In [ ]:
# Load required libraries
suppressPackageStartupMessages({
  library(shiny)
  library(ggplot2)
  library(dplyr)
})

# Set seed for reproducibility
set.seed(123)

# Define the number of days for the sample data
days <- 30

# Generate a sequence of dates for 30 consecutive days
dates <- seq.Date(from = as.Date("2023-01-01"), by = "day", length.out = days)

# Simulate daily Visits with random values between 200 and 700
Visits <- round(runif(days, min = 200, max = 700))

# Simulate Sales as a function of Visits with some random conversion rate (between 0.1 and 0.3)
Sales <- round(Visits * runif(days, min = 0.1, max = 0.3))

# Calculate Monthly Growth Rate (assuming a starting point of 100)
Growth <- c(100, cumsum(diff(Sales)))

# Create a data frame with the sample data
sample_data <- data.frame(
  Date = dates,
  Visits = Visits,
  Sales = Sales,
  Growth = Growth,
  stringsAsFactors = FALSE
)

# Define UI for the Shiny app
ui <- fluidPage(
  titlePanel("📊 Shiny App: Customizable Data Dashboard"),
  
  sidebarLayout(
    sidebarPanel(
      sliderInput("days", "Select range of days:", 
                  min = 1, max = nrow(sample_data), 
                  value = c(1, nrow(sample_data)), 
                  step = 1, 
                  animate = TRUE),
      selectInput("metric", "Select Metric:", choices = c("Visits", "Sales", "Growth")),
      checkboxInput("showTrend", "Show Trend Line", value = TRUE),
      br(),
      h4("Subset Summary"),
      textOutput("subsetCount"),
      br(),
      fluidRow(
        column(
          6,
          h6("First 5 Rows"),
          tableOutput("firstFive")
        ),
        column(
          6,
          h6("Last 5 Rows"),
          tableOutput("lastFive")
        )
      )
    ),
    
    mainPanel(
      plotOutput("barPlot"),
      plotOutput("scatterPlot")
    )
  )
)

# Define server logic
server <- function(input, output) {
  
  # Reactive data based on the selected range of days
  dataSubset <- reactive({
    start_index <- input$days[1]
    end_index <- input$days[2]
    sample_data[start_index:end_index, ]
  })
  
  # Render text output for the total number of rows in the reactive subset
  output$subsetCount <- renderText({
    paste("The subset has", nrow(dataSubset()), "rows.")
  })
  
  # Render a table for the first 5 rows
  output$firstFive <- renderTable({
    head(dataSubset(), 5)  # First 5 rows
  }, rownames = FALSE)
  
  # Render a table for the last 5 rows
  output$lastFive <- renderTable({
    tail(dataSubset(), 5)  # Last 5 rows
  }, rownames = FALSE)
  
  # Render bar plot for selected metric
  output$barPlot <- renderPlot({
    ggplot(dataSubset(), aes(x = Date, y = .data[[input$metric]])) +
      geom_bar(stat = "identity", fill = "steelblue") +
      labs(title = paste("Daily", input$metric, "Over Time"), x = "Date", y = input$metric) +
      theme_minimal() +
      theme(axis.text.x = element_text(angle = 45, hjust = 1))
  })
  
  # Render scatter plot for Sales vs Visits
  output$scatterPlot <- renderPlot({
    p <- ggplot(dataSubset(), aes(x = Visits, y = Sales)) +
      geom_point(color = "red", size = 3) +
      labs(title = "Sales vs Visits", x = "Number of Visits", y = "Number of Sales") +
      theme_minimal()
    
    if (input$showTrend) {
      p <- p + geom_smooth(method = "lm", color = "blue", se = FALSE, formula = y ~ x)
    }
    
    p
  })
}

# Run the Shiny app
shinyApp(ui = ui, server = server)



Listening on http://127.0.0.1:6846

